In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil.parser
import re

In [ ]:
# El siguiente comando significa que la salida de varios comandos en una celda se emitirá de una vez
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# El siguiente comando le dice a jupyter que muestre hasta 80 columnas, esto mantiene todo visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

# Muestra las figuras en el notebook
%matplotlib inline

In [ ]:
#BUSCAR EL ARCHIVO A CARGAR. la carpeta "data" debe estar en el mismo folder del proyecto creado en Azure. Experiment11.csv es el archivo a leer
df = pd.read_csv("data/Experiment1.csv")

In [ ]:
#ANALISIS EXPLORATORIO DE DATOS
#En la celda de abajo, tomamos una muestra aleatoria de 2 filas para tener una idea de los datos.
df.sample(n=2)
# Aquí están todas las columnas.
df.columns.tolist()
# Encuentra el dtype, también conocido como tipo de datos, para una columna
df['id_number'].dtype
#Muestra la tabla completa de datos
df.head()
#crear nueva columna con la suma de dos de estas
df['new'] = df['W'] + df['Y']
#eliminando columnas
df.drop('new',axis=1) #axis=0 index #inplace = True
#eliminar fila "E"
df.drop('E',axis=0)
#cambiar nombre a columna
names = df.columns.tolist()
names[names.index('two')] = 'new_name'
df.columns = names
#cambiar nombre a varias columnas (ej: de ""$a" cambiar a "a")
df.rename({'$a':'a', '$b':'b', '$c':'c', '$d':'d', '$e':'e'}, axis='columns')

In [ ]:
#Detectar si faltan datos 
#Crea una nueva función:
def num_missing(x):
    return sum(x.isnull())

#Aplicándola por columna:
print("Valores faltantes por columna:")
## Comprueba cuántos faltan por columna y luego verifica cuáles tienen los valores faltantes
print(df.apply(num_missing, axis=0).where(lambda x : x != 0).dropna().head(20)) 
#axis=0 Define que la función se aplica en cada columna.
print(f'Hay {df.shape[0]} observaciones y {df.shape[1]} características')
#ver cuantas observaciones (datos) y caracteristicas (columnas) hay

In [ ]:
#El comando "describe" a continuación proporciona estadísticas de resumen clave para cada columna numérica.
df.describe()

In [ ]:
#Extraer variables continuas del conjunto de datos
numeric_vars = df.select_dtypes(include=[np.number]).columns.tolist()
for variable in numeric_vars:
    print(variable)
#Graficar el comportamiento de dos variables 'lender count' y 'loan amount'
ax = sns.regplot(x='lender_count', y='loan_amount', data=df)

# Vamos a elegir sólo un par de columnas para examinar:
columns = ['loan_amount', 'funded_amount',  'status']
num_df = df[columns]

# Quita las filas de NaN para que Seaborn pueda graficar
num_df = num_df.dropna(axis=0, how='any')

# Crea el diagrama de dispersión y colorea los puntos de datos por su estado.
sns.pairplot(num_df, hue='status');
#mostrar a tabla para las tres columnas seleccionadas
num_df

#Histograma de columna 'loan amount'
sns.distplot(df['loan_amount'].dropna(axis = 0));
# Echemos un vistazo a los menores de 5K ya que estan mas acumulados acá
small_loans_df = df[(df['loan_amount'] < 5000)]
sns.distplot(small_loans_df['loan_amount']);

#Graficos de barra de variable 'loan amount' por 'sector' segun el promedio
p = sns.barplot(x='sector', y = 'loan_amount', data=df, estimator=np.mean);
p.set(title='Average loan amount by sector')
p.set_xticklabels(p.get_xticklabels(), rotation=-45);
#Graficos de barra de variable 'loan amount' por 'sector' segun la suma total de datos del sector
p = sns.barplot(x='sector', y = 'loan_amount', data=df, estimator=np.sum);
p.set(title='Total loan amount by sector')
p.set_xticklabels(p.get_xticklabels(), rotation=-45);

#Diagramas de caja para el sector retail
df_retail = df[df.sector=='Retail']
df_retail.head()
#Arriba se prepara la tabla de datos y abajo se muestra la grafica
sector = 'Retail'
df_retail = df[df.sector==sector]
p = sns.boxplot(x='sector', 
                y='loan_amount',
                data=df_retail);
p.set(title = f'Loan amounts for {sector}');
p.set_xticklabels(p.get_xticklabels(), rotation=-45);

#Graficar una variable segun el tiempo
# Convertir la fecha publicada en un objeto datetime
time_column = 'funded_date'
df[time_column] = pd.to_datetime(df[time_column])

# Volver a muestrear la fecha en intervalos mensuales, tomando el promedio de loan_amount
# Esto crea una matriz donde el índice es la marca de tiempo y el valor es el promedio del monto del préstamo
time_data = df.resample('M', on=time_column)['loan_amount'].mean().fillna(0)
fig, ax = plt.subplots(figsize=(15,8))
ax.plot(time_data)
plt.title('Promedio de loan_amount en el tiempo');
# Volver a muestrear la fecha en intervalos semanales, tomando el promedio de loan_amount
# Esto crea una matriz donde el índice es la marca de tiempo y el valor es el promedio del monto del préstamo
time_data = df.resample('7D', on=time_column)['loan_amount'].mean().fillna(0)
fig, ax = plt.subplots(figsize=(15,8))
ax.plot(time_data)
plt.title('Mean loan_amount over time');

In [ ]:
#INGENIERIA DE CARACTERISTICAS
#eliminar todas las caracteristicas que tengan una sola observacion
for col in df.columns:
    if df[col].unique().size==1:
        print("Dropping column: {0}".format(col))
        df = df.drop(col, axis=1)
#convertir strings a datos de fecha y hora. Todas las columnas que contengan '_date' (en otro conjunto de datos debería tener otro nombre) seran convertidas.
for col in [c for c in df.columns if "_date" in c]:
    if "_date" in col:
        df[col] = pd.to_datetime(df[col])
# This is temporally commented out if we really want to date based analysis

# ##  posted date features
# df['posted_year']=df['posted_date'].dt.year
# df['posted_month']=df['posted_date'].dt.month

# ## Time to fund is the funded date minus the posted date
# ## we add these fields because the homework question in the next notebook involves predicting time to fund
# df['time_to_fund'] =df['funded_date'] - df['posted_date']
# df['days_to_fund'] = df['time_to_fund'].dt.days

# # expiration date features
# ## Time to expiration is the expiration date minus the Posted Date
# df['time_to_expire_date'] =df['planned_expiration_date'] - df['posted_date']
# df['days_to_expire'] = df['time_to_expire_date'].dt.days

#get_dtype_counts para ver qué tipos de datos existen en el dataframe
df.get_dtype_counts()
#select_dtypes para ver todas las columnas con dtype == object
df.select_dtypes(include=[object])
#Tal vez la longitud de la descripción del texto podría ayudarnos para revisar la variable 'description length'
df['description_length']  = df.description.str.len()
df['description_length'].tail()

#Limpieza de datos
#repaso rápido de los tipos de datos en nuestro dataframe
df.get_dtype_counts()
#creemos listas de todas las columnas para tipos de datos específicos (clumna de tiempo: datatime64, etc; columan strings: objetc; columna numerica: todos menos los anteriores declarados en las doscolumnas anteriores)
time_columns = df.select_dtypes(include=['datetime64','timedelta64']).columns
str_columns = df.select_dtypes(include=[object]).columns
numeric_columns = df.select_dtypes(exclude=[object,'datetime64','timedelta64']).columns
#ver cuantas observaciones faltan en la columna de tiempo
f[time_columns].isnull().sum()[df[time_columns].isnull().sum()>0]
#Con los datos faltantes, siempre se debe verificar si hay una diferencia sistémica entre las observaciones con y sin datos faltantes.
#Con:
df[df['funded_date'].isnull()].describe()
#Sin los datos faltantes
df[~df['funded_date'].isnull()].describe()
#lo anterior se podria graficar para comparar diferecia sistemica
#Cree columnas que indiquen si faltan datos o no (solo con datos numericos):
for col in numeric_columns:
    df[col+'_na'] = pd.isnull(df[col])
#Impute los datos faltantes con la media
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
#Ahora que hemos terminado de crear todas nuestras características, podemos continuar guardando nuestros resultados en un nuevo csv.
df.to_csv("../data/loans_additional_features.csv", index=False) 
